This notebook demonstrates how to use the `QISKit ACQUA` library to invoke a specific algorithm and process the result.

Further information is available for the algorithms in the github repo qiskit_acqua/readme.md

In [1]:
import paths
from qiskit_acqua import Operator, run_algorithm
from qiskit_acqua.input import get_input_instance

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from a previously computed Hamiltonian for simplicity

In [2]:
pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.052373245772859}, "label": "II"},
              {"coeff": {"imag": 0.0, "real": 0.39793742484318045}, "label": "ZI"},
              {"coeff": {"imag": 0.0, "real": -0.39793742484318045}, "label": "IZ"},
              {"coeff": {"imag": 0.0, "real": -0.01128010425623538}, "label": "ZZ"},
              {"coeff": {"imag": 0.0, "real": 0.18093119978423156}, "label": "XX"}
              ]
}

qubitOp = Operator.load_from_dict(pauli_dict)

We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. Now the complete params can be passed to the algorithm and run. The result is a dictionary.

In [3]:
algorithm_cfg = {
    'name': 'ExactEigensolver',
}

params = {
    'algorithm': algorithm_cfg,
    'backend': {'name': 'local_statevector_simulator'}
}
algo_input = get_input_instance('EnergyInput')
algo_input.qubit_op = qubitOp
result = run_algorithm(params,algo_input)
print(result)

{'eigvals': array([-1.85727503+2.52764448e-17j]), 'eigvecs': array([[-5.55111512e-17-3.19839641e-17j,  7.22856695e-01+6.81936898e-01j,
        -8.11307233e-02-7.65380388e-02j, -2.22044605e-16+1.11022302e-16j]]), 'energy': -1.8572750302023817, 'wavefunction': array([[-5.55111512e-17-3.19839641e-17j,  7.22856695e-01+6.81936898e-01j,
        -8.11307233e-02-7.65380388e-02j, -2.22044605e-16+1.11022302e-16j]]), 'energies': array([-1.85727503])}


Now we want VQE and so change it and add its other configuration parameters. VQE also needs and optimizer and variational form. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

In [4]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'L_BFGS_B',
    'maxfun': 1000
}

var_form_cfg = {
    'name': 'RYRZ',
    'depth': 3,
    'entanglement': 'linear'
}

params = {
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg,
    'backend': {'name': 'local_statevector_simulator'}
}

result = run_algorithm(params,algo_input)
print(result)

{'eigvals': array([-1.85727503]), 'opt_params': array([ 1.80302537, -0.62015797, -2.41496323,  1.86977747, -2.47188931,
       -1.33812977,  0.60722118, -1.51702795, -0.76083903,  0.20803653,
       -2.04691407,  1.02412708, -0.7730152 , -2.49487229,  0.6355504 ,
        1.35011213]), 'eigvecs': array([[-1.07131905e-06-8.98032927e-07j,  9.93611028e-01-1.72361550e-02j,
        -1.11518161e-01+1.93367006e-03j, -1.53048025e-06-1.81372833e-06j]]), 'energy': -1.8572750301923562, 'eval_count': 255, 'eval_time': 4.485584735870361}
